# LangGraph Complete Tutorial: Building a Customer Support Agent System

Welcome to this comprehensive guide to LangGraph! We'll build a real-world customer support system that progressively demonstrates all core LangGraph capabilities.

## Prerequisites

1. **API Keys**: Create a `.env` file in your project root with your API keys:
   ```
   OPENAI_API_KEY=your-openai-key-here
   ```

## What We're Building

Imagine you're tasked with creating an AI-powered customer support system for a tech company. This system needs to:
- Route tickets to the right department
- Remember previous interactions
- Escalate to humans when needed
- Handle multiple agents working together
- And much more!

Let's start our journey!

## Setup and Installation

First, let's install the necessary packages:

In [87]:
# Install required packages
!pip install langgraph langchain langchain-openai langchain-community python-dotenv colorama -q

# If you encounter issues with SqliteSaver, try installing with extras:
# !pip install "langgraph[sqlite]"

In [89]:
# Import necessary libraries
import os
import sys
import json
import asyncio
from typing import TypedDict, Annotated, List, Dict, Any, Optional, Literal
from datetime import datetime
from colorama import Fore, Style, init
from IPython.display import display, Markdown, clear_output
import time

# Load environment variables from project root
from dotenv import load_dotenv

# Find project root and load .env
try:
    # Assumes the notebook is in 'labs/Day_01_.../' or similar structure
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    # Fallback for different execution environments
    project_root = os.path.abspath(os.path.join(os.getcwd()))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Load .env file from project root
dotenv_path = os.path.join(project_root, '.env')
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)
    print(f"✅ Loaded .env from: {dotenv_path}")
else:
    print(f"⚠️  No .env file found at: {dotenv_path}")
    print("Please create a .env file with your OPENAI_API_KEY")

# LangGraph imports
from langgraph.graph import StateGraph, Graph, END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.errors import NodeInterrupt

# Checkpoint imports - with fallback options
try:
    from langgraph.checkpoint.sqlite import SqliteSaver
except ImportError:
    try:
        from langgraph.checkpoint import SqliteSaver
    except ImportError:
        print("Warning: SqliteSaver not found. Using MemorySaver as fallback.")
        SqliteSaver = None

from langgraph.checkpoint.memory import MemorySaver

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate

# Initialize colorama for colored output
init(autoreset=True)

# Helper function to create checkpointer
def create_checkpointer(db_path=":memory:"):
    """Create appropriate checkpointer based on availability"""
    if SqliteSaver:
        return SqliteSaver.from_conn_string(db_path)
    else:
        return MemorySaver()

✅ Loaded .env from: /Users/agaleana/Documents/AI_Driven_Software_Engineering/.env


In [91]:
# Verify setup
print("🔍 Checking setup...\n")

# Check API key
api_key = os.environ.get("OPENAI_API_KEY", "")
if api_key:
    print("✅ OpenAI API key loaded successfully")
else:
    print("❌ OpenAI API key not found!")
    print("Please create a .env file in your project root with:")
    print("OPENAI_API_KEY=your-actual-api-key")

# Check LangGraph imports
try:
    from langgraph.graph import StateGraph
    print("✅ LangGraph imports successful")
except ImportError as e:
    print(f"❌ LangGraph import failed: {e}")

# Check SqliteSaver
if SqliteSaver:
    print("✅ SqliteSaver available")
else:
    print("⚠️  SqliteSaver not available - using MemorySaver")

print("\n🚀 Ready to start!")

🔍 Checking setup...

✅ OpenAI API key loaded successfully
✅ LangGraph imports successful
⚠️  SqliteSaver not available - using MemorySaver

🚀 Ready to start!


## Chapter 1: Basic Graph Structure

Let's start with the foundation - understanding what a graph is in LangGraph. Think of it as a flowchart where each box (node) performs a specific task, and arrows (edges) show how data flows between them.

In [93]:
# Define our state structure - this is like a shared notebook that all nodes can read and write to
class TicketState(TypedDict):
    """State that flows through our support ticket graph"""
    ticket_id: str
    customer_message: str
    department: Optional[str]
    priority: Optional[str]
    resolved: bool
    messages: List[BaseMessage]  # Chat history


# Our first simple node - analyzes the ticket
def analyze_ticket(state: TicketState) -> TicketState:
    """Analyzes incoming ticket and determines department and priority"""
    print(f"{Fore.CYAN}🔍 Analyzing ticket...{Style.RESET_ALL}")
    
    # In a real system, this would use AI to analyze the message
    message = state["customer_message"].lower()
    
    # Simple keyword-based routing
    if "password" in message or "login" in message:
        state["department"] = "security"
    elif "payment" in message or "billing" in message:
        state["department"] = "billing"
    elif "bug" in message or "error" in message:
        state["department"] = "technical"
    else:
        state["department"] = "general"
    
    # Determine priority
    if "urgent" in message or "asap" in message:
        state["priority"] = "high"
    else:
        state["priority"] = "normal"
    
    print(f"{Fore.GREEN}✓ Routed to {state['department']} department with {state['priority']} priority{Style.RESET_ALL}")
    return state


# Create our first graph
def create_basic_graph():
    """Creates a simple ticket routing graph"""
    # Initialize the graph with our state type
    graph = StateGraph(TicketState)
    
    # Add our analyze node
    graph.add_node("analyze", analyze_ticket)
    
    # Set the entry point
    graph.set_entry_point("analyze")
    
    # Connect analyze to END
    graph.add_edge("analyze", END)
    
    # Compile the graph
    return graph.compile()


# Test our basic graph
print("🚀 Testing Basic Graph:\n")
basic_graph = create_basic_graph()

# Create a test ticket
test_ticket = {
    "ticket_id": "TICKET-001",
    "customer_message": "URGENT: I can't login to my account!",
    "resolved": False,
    "messages": []
}

# Run the graph
result = basic_graph.invoke(test_ticket)
print(f"\n📋 Result: Department={result['department']}, Priority={result['priority']}")

🚀 Testing Basic Graph:

🔍 Analyzing ticket...
✓ Routed to security department with high priority

📋 Result: Department=security, Priority=high


## Chapter 2: Adding Models (LLMs)

Now let's make our agent smarter by adding a language model. This transforms our simple keyword matching into intelligent understanding.

**Note**: We're constructing messages directly rather than using ChatPromptTemplate to avoid issues with mixing f-strings and template placeholders. This approach is cleaner when we need to include dynamic state values in our prompts.

In [96]:
# Initialize our LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


def analyze_with_llm(state: TicketState) -> TicketState:
    """Uses LLM to intelligently analyze tickets"""
    print(f"{Fore.CYAN}🤖 AI analyzing ticket...{Style.RESET_ALL}")
    
    # Create a prompt for the LLM
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a customer support ticket analyzer. 
        Analyze the customer message and determine:
        1. Department: security, billing, technical, or general
        2. Priority: high, medium, or low
        3. A brief summary of the issue
        
        Respond in JSON format:
        {"department": "...", "priority": "...", "summary": "..."}
        """),
        ("human", "{message}")
    ])
    
    # Get LLM response
    chain = prompt | llm
    response = chain.invoke({"message": state["customer_message"]})
    
    # Parse the response
    try:
        analysis = json.loads(response.content)
        state["department"] = analysis["department"]
        state["priority"] = analysis["priority"]
        
        # Add the analysis to our message history
        state["messages"].append(AIMessage(content=f"Analysis: {analysis['summary']}"))
        
        print(f"{Fore.GREEN}✓ AI Analysis complete:{Style.RESET_ALL}")
        print(f"  Department: {analysis['department']}")
        print(f"  Priority: {analysis['priority']}")
        print(f"  Summary: {analysis['summary']}")
    except:
        # Fallback to simple analysis
        print(f"{Fore.YELLOW}⚠ Falling back to simple analysis{Style.RESET_ALL}")
        return analyze_ticket(state)
    
    return state


# Create an enhanced graph with LLM
def create_llm_graph():
    graph = StateGraph(TicketState)
    graph.add_node("analyze", analyze_with_llm)
    graph.set_entry_point("analyze")
    graph.add_edge("analyze", END)
    return graph.compile()


# Test the LLM-powered graph
print("\n🤖 Testing LLM-Powered Graph:\n")
llm_graph = create_llm_graph()

test_ticket_complex = {
    "ticket_id": "TICKET-002",
    "customer_message": "I was charged twice for my subscription last month. This is really frustrating and I need this fixed immediately!",
    "resolved": False,
    "messages": []
}

result = llm_graph.invoke(test_ticket_complex)


🤖 Testing LLM-Powered Graph:

🤖 AI analyzing ticket...


KeyError: 'Input to ChatPromptTemplate is missing variables {\'"department"\'}.  Expected: [\'"department"\', \'message\'] Received: [\'message\']\nNote: if you intended {"department"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{"department"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '

## Chapter 3: Streaming - Real-time Updates

In a real support system, we want to see updates as they happen, not wait until everything is done. Streaming gives us real-time visibility into what our agent is doing.

In [ ]:
def respond_to_customer(state: TicketState) -> TicketState:
    """Generate a response to the customer"""
    print(f"{Fore.CYAN}💬 Generating response...{Style.RESET_ALL}")
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"""You are a helpful customer support agent. 
        The ticket has been routed to the {state['department']} department with {state['priority']} priority.
        Provide a helpful initial response to acknowledge their issue."""),
        ("human", "{message}")
    ])
    
    chain = prompt | llm
    response = chain.invoke({"message": state["customer_message"]})
    state["messages"].append(response)
    
    return state


def create_streaming_graph():
    """Create a graph that supports streaming"""
    graph = StateGraph(TicketState)
    
    # Add multiple nodes to see streaming in action
    graph.add_node("analyze", analyze_with_llm)
    graph.add_node("respond", respond_to_customer)
    
    # Connect them
    graph.set_entry_point("analyze")
    graph.add_edge("analyze", "respond")
    graph.add_edge("respond", END)
    
    return graph.compile()


# Demonstrate streaming
print("\n📡 Testing Streaming:\n")
streaming_graph = create_streaming_graph()

# Stream the execution
print("Streaming updates as they happen:")
print("-" * 50)

for event in streaming_graph.stream(test_ticket_complex):
    # Each event shows what node was executed and its output
    for node, output in event.items():
        print(f"\n{Fore.BLUE}[{node}]{Style.RESET_ALL} completed")
        if node == "respond" and output.get("messages"):
            # Show the generated response
            last_message = output["messages"][-1]
            print(f"\n{Fore.GREEN}Response to customer:{Style.RESET_ALL}")
            print(f"{last_message.content}")
    time.sleep(0.5)  # Simulate real-time updates

## Chapter 4: Persistence - Saving State

What happens when our system crashes? Or when we need to handle thousands of tickets? Persistence allows us to save and restore our agent's state.

In [ ]:
# Create a persistent graph with checkpointing
def create_persistent_graph():
    """Create a graph with persistence capabilities"""
    graph = StateGraph(TicketState)
    
    graph.add_node("analyze", analyze_with_llm)
    graph.add_node("respond", respond_to_customer)
    
    graph.set_entry_point("analyze")
    graph.add_edge("analyze", "respond")
    graph.add_edge("respond", END)
    
    # Add persistence with checkpointer
    checkpointer = create_checkpointer()
    
    return graph.compile(checkpointer=checkpointer)


print("\n💾 Testing Persistence:\n")
persistent_graph = create_persistent_graph()

# Process a ticket with a thread ID (this allows us to resume later)
config = {"configurable": {"thread_id": "customer-123"}}

# First run
print("First execution:")
result1 = persistent_graph.invoke(test_ticket_complex, config=config)

# Now let's "resume" the conversation
print("\n\n🔄 Resuming conversation with same thread:")
followup_ticket = {
    "ticket_id": "TICKET-002-FOLLOWUP",
    "customer_message": "Thanks for the response. When can I expect my refund?",
    "resolved": False,
    "messages": result1["messages"]  # Continue with previous messages
}

# The graph remembers the previous state!
result2 = persistent_graph.invoke(followup_ticket, config=config)

# Show the conversation history
print("\n📜 Full conversation history:")
for i, msg in enumerate(result2["messages"]):
    print(f"\n[{i+1}] {type(msg).__name__}: {msg.content[:100]}...")

## Chapter 5: Tools - Giving Agents Abilities

Real support agents need to do more than just talk - they need to check orders, process refunds, and update systems. Tools give our agents these abilities.

In [ ]:
# Define tools that our agent can use
@tool
def check_order_status(order_id: str) -> str:
    """Check the status of a customer order"""
    # Simulate database lookup
    print(f"{Fore.YELLOW}🔍 Checking order {order_id}...{Style.RESET_ALL}")
    return f"Order {order_id} is currently being processed. Expected delivery: 3-5 business days."


@tool
def process_refund(order_id: str, amount: float) -> str:
    """Process a refund for a customer order"""
    print(f"{Fore.YELLOW}💰 Processing refund of ${amount} for order {order_id}...{Style.RESET_ALL}")
    return f"Refund of ${amount} has been initiated. It will appear in your account within 5-7 business days."


@tool
def escalate_to_human(reason: str) -> str:
    """Escalate the ticket to a human agent"""
    print(f"{Fore.RED}🚨 Escalating to human: {reason}{Style.RESET_ALL}")
    return "Your ticket has been escalated to a human agent who will contact you within 24 hours."


# Create a more sophisticated state that includes tool calls
class AdvancedTicketState(TypedDict):
    ticket_id: str
    customer_message: str
    department: Optional[str]
    priority: Optional[str]
    resolved: bool
    messages: List[BaseMessage]
    next_action: Optional[str]  # What to do next


def agent_with_tools(state: AdvancedTicketState) -> AdvancedTicketState:
    """An agent that can use tools to help customers"""
    print(f"{Fore.CYAN}🤖 Agent processing with tools...{Style.RESET_ALL}")
    
    # Create an agent that can use tools
    tools = [check_order_status, process_refund, escalate_to_human]
    llm_with_tools = llm.bind_tools(tools)
    
    # Prepare the conversation
    messages = [
        SystemMessage(content="""You are a helpful customer support agent with access to tools.
        Use the appropriate tools to help the customer.
        Always be polite and professional."""),
        HumanMessage(content=state["customer_message"])
    ]
    
    # Get the agent's response
    response = llm_with_tools.invoke(messages)
    state["messages"].append(response)
    
    # Check if the agent wants to use tools
    if response.tool_calls:
        state["next_action"] = "use_tools"
    else:
        state["next_action"] = "respond"
    
    return state


def create_tool_graph():
    """Create a graph with tool capabilities"""
    graph = StateGraph(AdvancedTicketState)
    
    # Add nodes
    graph.add_node("agent", agent_with_tools)
    graph.add_node("tools", ToolNode([check_order_status, process_refund, escalate_to_human]))
    
    # Add conditional routing
    def route_next(state: AdvancedTicketState) -> str:
        if state["next_action"] == "use_tools":
            return "tools"
        return END
    
    graph.set_entry_point("agent")
    graph.add_conditional_edges("agent", route_next)
    graph.add_edge("tools", "agent")  # After tools, go back to agent
    
    return graph.compile()


# Test the tool-enabled agent
print("\n🛠️ Testing Agent with Tools:\n")
tool_graph = create_tool_graph()

# Test with a refund request
refund_request = {
    "ticket_id": "TICKET-003",
    "customer_message": "I need a refund for order #12345. I was charged $99.99 twice!",
    "resolved": False,
    "messages": [],
    "next_action": None
}

# Stream the execution to see tools in action
for event in tool_graph.stream(refund_request):
    for node, output in event.items():
        print(f"\n{Fore.BLUE}[{node}]{Style.RESET_ALL} executed")
        if output.get("messages"):
            last_msg = output["messages"][-1]
            if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
                print("Tool calls requested:", [tc["name"] for tc in last_msg.tool_calls])

## Chapter 6: Human-in-the-Loop

Some decisions are too important for AI alone. Human-in-the-loop patterns allow us to pause execution and wait for human approval before proceeding.

In [ ]:
def requires_approval(state: AdvancedTicketState) -> AdvancedTicketState:
    """Check if an action requires human approval"""
    print(f"{Fore.YELLOW}🤔 Checking if approval needed...{Style.RESET_ALL}")
    
    # Check the last message for high-value actions
    last_message = state["messages"][-1] if state["messages"] else None
    
    if last_message and hasattr(last_message, "tool_calls"):
        for tool_call in last_message.tool_calls:
            # Refunds over $100 need approval
            if tool_call["name"] == "process_refund":
                amount = tool_call["args"].get("amount", 0)
                if amount > 100:
                    print(f"{Fore.RED}⚠️  High-value refund (${amount}) requires approval!{Style.RESET_ALL}")
                    # Interrupt execution for human approval
                    raise NodeInterrupt(f"Approval needed for ${amount} refund")
    
    return state


def create_human_in_loop_graph():
    """Create a graph with human approval workflow"""
    graph = StateGraph(AdvancedTicketState)
    
    graph.add_node("agent", agent_with_tools)
    graph.add_node("check_approval", requires_approval)
    graph.add_node("tools", ToolNode([check_order_status, process_refund, escalate_to_human]))
    
    def route_after_agent(state: AdvancedTicketState) -> str:
        if state["next_action"] == "use_tools":
            return "check_approval"
        return END
    
    graph.set_entry_point("agent")
    graph.add_conditional_edges("agent", route_after_agent)
    graph.add_edge("check_approval", "tools")
    graph.add_edge("tools", "agent")
    
    # Use memory saver for interrupts
    return graph.compile(checkpointer=MemorySaver())


print("\n👤 Testing Human-in-the-Loop:\n")
human_graph = create_human_in_loop_graph()

# Test with a high-value refund
high_value_refund = {
    "ticket_id": "TICKET-004",
    "customer_message": "I was incorrectly charged $250 for a premium subscription I never ordered. Please refund immediately!",
    "resolved": False,
    "messages": [],
    "next_action": None
}

config = {"configurable": {"thread_id": "high-value-001"}}

# First execution - will interrupt for approval
print("Initial execution:")
try:
    for event in human_graph.stream(high_value_refund, config=config):
        for node, output in event.items():
            print(f"\n{Fore.BLUE}[{node}]{Style.RESET_ALL} executed")
except Exception as e:
    print(f"\n{Fore.RED}Execution interrupted: {e}{Style.RESET_ALL}")
    print("\nWaiting for human approval...")
    
    # Simulate human approval
    input("\nPress Enter to approve the refund...")
    
    # Resume execution after approval
    print("\n✅ Approved! Resuming execution...")
    for event in human_graph.stream(None, config=config):
        for node, output in event.items():
            print(f"\n{Fore.BLUE}[{node}]{Style.RESET_ALL} executed")

## Chapter 7: Memory and Context

Good customer support remembers past interactions. Let's add long-term memory to our agent so it can provide personalized service.

In [ ]:
# Enhanced state with customer history
class CustomerMemoryState(TypedDict):
    ticket_id: str
    customer_id: str
    customer_message: str
    customer_history: List[Dict[str, Any]]  # Past interactions
    messages: List[BaseMessage]
    resolved: bool


# Simulate a customer database
CUSTOMER_DATABASE = {
    "CUST-001": {
        "name": "Alice Johnson",
        "tier": "Premium",
        "history": [
            {"date": "2024-01-15", "issue": "Login problems", "resolved": True},
            {"date": "2024-02-20", "issue": "Billing question", "resolved": True}
        ]
    },
    "CUST-002": {
        "name": "Bob Smith",
        "tier": "Basic",
        "history": []
    }
}


def load_customer_context(state: CustomerMemoryState) -> CustomerMemoryState:
    """Load customer history and context"""
    print(f"{Fore.CYAN}📚 Loading customer context...{Style.RESET_ALL}")
    
    customer_id = state["customer_id"]
    customer_data = CUSTOMER_DATABASE.get(customer_id, {})
    
    if customer_data:
        state["customer_history"] = customer_data["history"]
        print(f"✓ Loaded history for {customer_data['name']} ({customer_data['tier']} tier)")
        print(f"  Previous tickets: {len(customer_data['history'])}")
    else:
        state["customer_history"] = []
        print("  New customer - no history found")
    
    return state


def personalized_agent(state: CustomerMemoryState) -> CustomerMemoryState:
    """Agent that uses customer history for personalized responses"""
    print(f"{Fore.CYAN}🤖 Generating personalized response...{Style.RESET_ALL}")
    
    customer_data = CUSTOMER_DATABASE.get(state["customer_id"], {})
    
    # Build context from history
    history_context = ""
    if state["customer_history"]:
        history_context = "Previous interactions:\n"
        for interaction in state["customer_history"]:
            history_context += f"- {interaction['date']}: {interaction['issue']}\n"
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"""You are a customer support agent with access to customer history.
        Customer: {customer_data.get('name', 'Unknown')}
        Tier: {customer_data.get('tier', 'Basic')}
        {history_context}
        
        Provide personalized support based on their history and tier.
        Premium customers get priority treatment."""),
        ("human", "{message}")
    ])
    
    chain = prompt | llm
    response = chain.invoke({"message": state["customer_message"]})
    state["messages"].append(response)
    
    # Update customer history
    new_interaction = {
        "date": datetime.now().strftime("%Y-%m-%d"),
        "issue": state["customer_message"][:50] + "...",
        "resolved": state["resolved"]
    }
    CUSTOMER_DATABASE[state["customer_id"]]["history"].append(new_interaction)
    
    return state


def create_memory_graph():
    """Create a graph with customer memory"""
    graph = StateGraph(CustomerMemoryState)
    
    graph.add_node("load_context", load_customer_context)
    graph.add_node("personalized_response", personalized_agent)
    
    graph.set_entry_point("load_context")
    graph.add_edge("load_context", "personalized_response")
    graph.add_edge("personalized_response", END)
    
    return graph.compile()


print("\n🧠 Testing Memory and Context:\n")
memory_graph = create_memory_graph()

# Test with a returning premium customer
premium_customer_ticket = {
    "ticket_id": "TICKET-005",
    "customer_id": "CUST-001",
    "customer_message": "I'm having login issues again. This is the second time this year!",
    "customer_history": [],
    "messages": [],
    "resolved": False
}

result = memory_graph.invoke(premium_customer_ticket)
print(f"\n{Fore.GREEN}Response:{Style.RESET_ALL}")
print(result["messages"][-1].content)

## Chapter 8: Time Travel - Debugging and Analysis

When things go wrong, we need to understand what happened. Time travel lets us replay executions and see exactly what our agent did at each step.

**Note**: Since we're using `AdvancedTicketState` here, we need adapted versions of our functions that handle this state type.

In [ ]:
def create_debuggable_graph():
    """Create a graph with full debugging capabilities"""
    graph = StateGraph(AdvancedTicketState)
    
    # Add multiple nodes to track
    graph.add_node("analyze", analyze_with_llm)
    graph.add_node("route", lambda s: {**s, "department": s.get("department", "general")})
    graph.add_node("respond", respond_to_customer)
    
    graph.set_entry_point("analyze")
    graph.add_edge("analyze", "route")
    graph.add_edge("route", "respond")
    graph.add_edge("respond", END)
    
    # Use persistent checkpointing
    checkpointer = create_checkpointer()
    return graph.compile(checkpointer=checkpointer)


print("\n⏰ Testing Time Travel Debugging:\n")
debug_graph = create_debuggable_graph()

# Execute with detailed tracking
config = {"configurable": {"thread_id": "debug-session-001"}}

test_ticket = {
    "ticket_id": "TICKET-006",
    "customer_message": "My application keeps crashing when I try to export data.",
    "resolved": False,
    "messages": [],
    "next_action": None
}

# Run the graph
print("Executing graph with checkpointing...")
result = debug_graph.invoke(test_ticket, config=config)

# Now let's travel through time and see what happened
print("\n🔍 Replaying execution history:")
print("-" * 50)

# Get all states from the execution
states = []
for state in debug_graph.get_state_history(config):
    states.append(state)

# Show states in chronological order (reverse the list)
states.reverse()

for i, state in enumerate(states):
    print(f"\n📍 Step {i}: Node '{state.metadata.get('langgraph_node', 'unknown')}'")
    if state.values.get("department"):
        print(f"   Department: {state.values['department']}")
    if state.values.get("priority"):
        print(f"   Priority: {state.values['priority']}")
    print(f"   Messages: {len(state.values.get('messages', []))}")

# You can also replay from any point
print("\n🔄 Replaying from a specific checkpoint...")
if len(states) > 1:
    # Get state from before the last node
    replay_state = states[-2]
    print(f"Replaying from state before '{replay_state.metadata.get('langgraph_node')}'")
    
    # Modify and replay
    modified_input = {
        **replay_state.values,
        "customer_message": "Actually, it's a CRITICAL production issue!"
    }
    
    replay_result = debug_graph.invoke(modified_input, config={"configurable": {"thread_id": "replay-001"}})
    print(f"New priority after replay: {replay_result.get('priority')}")

## Chapter 9: Subgraphs - Modular Architecture

As our system grows, we need to organize it better. Subgraphs let us create modular, reusable components that can be composed together.

In [ ]:
# Create specialized subgraphs for different departments

def create_billing_subgraph():
    """Subgraph specifically for billing issues"""
    
    class BillingState(TypedDict):
        issue_type: str
        amount: Optional[float]
        action_taken: str
        messages: List[BaseMessage]
    
    def categorize_billing_issue(state: BillingState) -> BillingState:
        print(f"{Fore.YELLOW}💳 Categorizing billing issue...{Style.RESET_ALL}")
        # Simple categorization logic
        state["issue_type"] = "refund_request"
        state["amount"] = 99.99
        return state
    
    def process_billing_action(state: BillingState) -> BillingState:
        print(f"{Fore.YELLOW}💰 Processing billing action...{Style.RESET_ALL}")
        if state["issue_type"] == "refund_request":
            state["action_taken"] = f"Refund of ${state['amount']} initiated"
        return state
    
    graph = StateGraph(BillingState)
    graph.add_node("categorize", categorize_billing_issue)
    graph.add_node("process", process_billing_action)
    
    graph.set_entry_point("categorize")
    graph.add_edge("categorize", "process")
    graph.add_edge("process", END)
    
    return graph.compile()


def create_technical_subgraph():
    """Subgraph for technical support issues"""
    
    class TechnicalState(TypedDict):
        error_type: str
        severity: str
        solution: str
        messages: List[BaseMessage]
    
    def diagnose_issue(state: TechnicalState) -> TechnicalState:
        print(f"{Fore.CYAN}🔧 Diagnosing technical issue...{Style.RESET_ALL}")
        state["error_type"] = "application_crash"
        state["severity"] = "high"
        return state
    
    def provide_solution(state: TechnicalState) -> TechnicalState:
        print(f"{Fore.CYAN}💡 Providing solution...{Style.RESET_ALL}")
        state["solution"] = "Please try clearing your cache and updating to the latest version."
        return state
    
    graph = StateGraph(TechnicalState)
    graph.add_node("diagnose", diagnose_issue)
    graph.add_node("solve", provide_solution)
    
    graph.set_entry_point("diagnose")
    graph.add_edge("diagnose", "solve")
    graph.add_edge("solve", END)
    
    return graph.compile()


# Main graph that routes to subgraphs
class MainRoutingState(TypedDict):
    ticket_id: str
    customer_message: str
    department: str
    subgraph_result: Optional[Dict[str, Any]]
    messages: List[BaseMessage]


def create_main_graph_with_subgraphs():
    """Main graph that delegates to specialized subgraphs"""
    
    # Create subgraphs
    billing_graph = create_billing_subgraph()
    technical_graph = create_technical_subgraph()
    
    def route_to_department(state: MainRoutingState) -> MainRoutingState:
        print(f"{Fore.BLUE}🚦 Routing to department...{Style.RESET_ALL}")
        
        # Simple routing logic
        if "billing" in state["customer_message"].lower() or "refund" in state["customer_message"].lower():
            state["department"] = "billing"
        elif "error" in state["customer_message"].lower() or "crash" in state["customer_message"].lower():
            state["department"] = "technical"
        else:
            state["department"] = "general"
        
        print(f"  → Routed to {state['department']}")
        return state
    
    def handle_billing(state: MainRoutingState) -> MainRoutingState:
        print(f"{Fore.YELLOW}🏦 Handling in billing department...{Style.RESET_ALL}")
        # Prepare input for billing subgraph
        billing_input = {
            "issue_type": None,
            "amount": None,
            "action_taken": None,
            "messages": state["messages"]
        }
        
        # Execute billing subgraph
        result = billing_graph.invoke(billing_input)
        state["subgraph_result"] = result
        return state
    
    def handle_technical(state: MainRoutingState) -> MainRoutingState:
        print(f"{Fore.CYAN}⚙️ Handling in technical department...{Style.RESET_ALL}")
        # Prepare input for technical subgraph
        tech_input = {
            "error_type": None,
            "severity": None,
            "solution": None,
            "messages": state["messages"]
        }
        
        # Execute technical subgraph
        result = technical_graph.invoke(tech_input)
        state["subgraph_result"] = result
        return state
    
    def handle_general(state: MainRoutingState) -> MainRoutingState:
        print(f"{Fore.GREEN}📋 Handling in general support...{Style.RESET_ALL}")
        state["subgraph_result"] = {"response": "Thank you for contacting support. We'll help you shortly."}
        return state
    
    # Build main graph
    graph = StateGraph(MainRoutingState)
    
    # Add nodes
    graph.add_node("route", route_to_department)
    graph.add_node("billing", handle_billing)
    graph.add_node("technical", handle_technical)
    graph.add_node("general", handle_general)
    
    # Add conditional routing
    def select_department(state: MainRoutingState) -> str:
        return state["department"]
    
    graph.set_entry_point("route")
    graph.add_conditional_edges(
        "route",
        select_department,
        {
            "billing": "billing",
            "technical": "technical",
            "general": "general"
        }
    )
    
    # All departments lead to END
    graph.add_edge("billing", END)
    graph.add_edge("technical", END)
    graph.add_edge("general", END)
    
    return graph.compile()


print("\n🏗️ Testing Subgraphs:\n")
main_graph = create_main_graph_with_subgraphs()

# Test different types of tickets
test_tickets = [
    {
        "ticket_id": "TICKET-007",
        "customer_message": "I need a refund for my subscription",
        "messages": []
    },
    {
        "ticket_id": "TICKET-008",
        "customer_message": "The app crashes when I try to export",
        "messages": []
    }
]

for ticket in test_tickets:
    print(f"\n{'='*60}")
    print(f"Processing: {ticket['customer_message']}")
    print(f"{'='*60}")
    
    result = main_graph.invoke(ticket)
    print(f"\n✅ Result: {result.get('subgraph_result', {})}")

## Chapter 10: Multi-Agent Collaboration

Complex problems often need multiple specialists working together. Let's create a multi-agent system where different AI agents collaborate to solve customer issues.

In [ ]:
# Define specialized agents
class MultiAgentState(TypedDict):
    ticket_id: str
    customer_message: str
    analysis: Dict[str, Any]
    technical_assessment: Optional[str]
    business_impact: Optional[str]
    proposed_solution: Optional[str]
    final_response: Optional[str]
    messages: List[BaseMessage]


def analyst_agent(state: MultiAgentState) -> MultiAgentState:
    """Analyst agent that understands the problem"""
    print(f"{Fore.BLUE}👔 Analyst Agent: Analyzing problem...{Style.RESET_ALL}")
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a business analyst. Analyze the customer's issue and identify:
        1. The core problem
        2. Business impact
        3. Urgency level
        Respond in JSON format."""),
        ("human", "{message}")
    ])
    
    chain = prompt | llm
    response = chain.invoke({"message": state["customer_message"]})
    
    try:
        state["analysis"] = json.loads(response.content)
        print(f"  ✓ Analysis complete: {state['analysis'].get('core_problem', 'Unknown')}")
    except:
        state["analysis"] = {"core_problem": "Unable to parse", "impact": "unknown", "urgency": "medium"}
    
    return state


def technical_agent(state: MultiAgentState) -> MultiAgentState:
    """Technical agent that assesses technical aspects"""
    print(f"{Fore.CYAN}🔧 Technical Agent: Evaluating technical aspects...{Style.RESET_ALL}")
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"""You are a technical expert. Based on this analysis:
        {json.dumps(state['analysis'], indent=2)}
        
        Provide a technical assessment and identify potential root causes."""),
        ("human", "{message}")
    ])
    
    chain = prompt | llm
    response = chain.invoke({"message": state["customer_message"]})
    state["technical_assessment"] = response.content
    
    print(f"  ✓ Technical assessment completed")
    return state


def solution_architect(state: MultiAgentState) -> MultiAgentState:
    """Solution architect that designs the solution"""
    print(f"{Fore.GREEN}🏗️ Solution Architect: Designing solution...{Style.RESET_ALL}")
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"""You are a solution architect. Based on:
        Analysis: {json.dumps(state['analysis'], indent=2)}
        Technical Assessment: {state['technical_assessment']}
        
        Design a comprehensive solution that addresses the root cause."""),
        ("human", "{message}")
    ])
    
    chain = prompt | llm
    response = chain.invoke({"message": state["customer_message"]})
    state["proposed_solution"] = response.content
    
    print(f"  ✓ Solution designed")
    return state


def communication_specialist(state: MultiAgentState) -> MultiAgentState:
    """Communication specialist that crafts the customer response"""
    print(f"{Fore.MAGENTA}💬 Communication Specialist: Crafting response...{Style.RESET_ALL}")
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"""You are a communication specialist. Based on all the expert input:
        Analysis: {json.dumps(state['analysis'], indent=2)}
        Technical Assessment: {state['technical_assessment']}
        Solution: {state['proposed_solution']}
        
        Craft a clear, empathetic response for the customer that:
        1. Acknowledges their issue
        2. Explains what we understand
        3. Provides the solution
        4. Sets clear expectations"""),
        ("human", "{message}")
    ])
    
    chain = prompt | llm
    response = chain.invoke({"message": state["customer_message"]})
    state["final_response"] = response.content
    state["messages"].append(response)
    
    print(f"  ✓ Customer response prepared")
    return state


def supervisor_agent(state: MultiAgentState) -> str:
    """Supervisor that coordinates the agents"""
    print(f"{Fore.YELLOW}👨‍💼 Supervisor: Determining next agent...{Style.RESET_ALL}")
    
    # Determine which agent should work next
    if not state.get("analysis"):
        return "analyst"
    elif not state.get("technical_assessment"):
        return "technical"
    elif not state.get("proposed_solution"):
        return "architect"
    elif not state.get("final_response"):
        return "communicator"
    else:
        return "end"


def create_multi_agent_graph():
    """Create a multi-agent collaboration graph"""
    graph = StateGraph(MultiAgentState)
    
    # Add all agent nodes
    graph.add_node("analyst", analyst_agent)
    graph.add_node("technical", technical_agent)
    graph.add_node("architect", solution_architect)
    graph.add_node("communicator", communication_specialist)
    
    # Add conditional edges based on supervisor decisions
    graph.add_conditional_edges(
        "analyst",
        supervisor_agent,
        {"technical": "technical", "end": END}
    )
    
    graph.add_conditional_edges(
        "technical",
        supervisor_agent,
        {"architect": "architect", "end": END}
    )
    
    graph.add_conditional_edges(
        "architect",
        supervisor_agent,
        {"communicator": "communicator", "end": END}
    )
    
    graph.add_edge("communicator", END)
    
    # Set entry point
    graph.set_entry_point("analyst")
    
    return graph.compile()


print("\n👥 Testing Multi-Agent Collaboration:\n")
multi_agent_graph = create_multi_agent_graph()

# Test with a complex issue
complex_issue = {
    "ticket_id": "TICKET-009",
    "customer_message": """Our team can't access the dashboard since yesterday's update. 
    We're losing valuable time and this is affecting our quarterly reporting. 
    Multiple users across different departments are experiencing this.""",
    "messages": []
}

# Stream the multi-agent collaboration
print("🎭 Agents collaborating on the issue:")
print("=" * 60)

for event in multi_agent_graph.stream(complex_issue):
    time.sleep(1)  # Simulate thinking time

# Show final response
print(f"\n{Fore.GREEN}📨 Final Response to Customer:{Style.RESET_ALL}")
print("-" * 60)
final_state = multi_agent_graph.invoke(complex_issue)
print(final_state["final_response"])

## Chapter 11: Durable Execution

In production, our agents need to handle failures gracefully. Durable execution ensures that if something goes wrong, we can recover without losing progress.

In [ ]:
import random

class DurableState(TypedDict):
    ticket_id: str
    steps_completed: List[str]
    retry_count: int
    final_result: Optional[str]


def unreliable_api_call(state: DurableState) -> DurableState:
    """Simulates an unreliable external API call"""
    print(f"{Fore.YELLOW}🌐 Calling external API (attempt {state['retry_count'] + 1})...{Style.RESET_ALL}")
    
    # Simulate 50% failure rate
    if random.random() < 0.5 and state["retry_count"] < 2:
        state["retry_count"] += 1
        print(f"{Fore.RED}❌ API call failed! Will retry...{Style.RESET_ALL}")
        raise Exception("API temporarily unavailable")
    
    print(f"{Fore.GREEN}✅ API call successful!{Style.RESET_ALL}")
    state["steps_completed"].append("api_call")
    state["retry_count"] = 0
    return state


def process_data(state: DurableState) -> DurableState:
    """Process data after API call"""
    print(f"{Fore.CYAN}📊 Processing data...{Style.RESET_ALL}")
    state["steps_completed"].append("data_processing")
    return state


def generate_report(state: DurableState) -> DurableState:
    """Generate final report"""
    print(f"{Fore.GREEN}📄 Generating report...{Style.RESET_ALL}")
    state["steps_completed"].append("report_generation")
    state["final_result"] = "Report generated successfully!"
    return state


def create_durable_graph():
    """Create a graph with durable execution capabilities"""
    graph = StateGraph(DurableState)
    
    # Add nodes
    graph.add_node("api_call", unreliable_api_call)
    graph.add_node("process", process_data)
    graph.add_node("report", generate_report)
    
    # Define retry logic
    def should_retry(state: DurableState) -> str:
        if "api_call" not in state["steps_completed"] and state["retry_count"] < 3:
            return "api_call"
        elif "api_call" in state["steps_completed"]:
            return "process"
        else:
            return "report"  # Give up and generate error report
    
    # Set up flow with retry logic
    graph.set_entry_point("api_call")
    graph.add_conditional_edges("api_call", should_retry)
    graph.add_edge("process", "report")
    graph.add_edge("report", END)
    
    # Use persistent checkpointer for durability
    checkpointer = create_checkpointer()
    return graph.compile(checkpointer=checkpointer)


print("\n🛡️ Testing Durable Execution:\n")
durable_graph = create_durable_graph()

# Test with automatic retries
durable_state = {
    "ticket_id": "TICKET-010",
    "steps_completed": [],
    "retry_count": 0,
    "final_result": None
}

config = {"configurable": {"thread_id": "durable-001"}}

# Run with retry handling
print("Starting durable execution with potential failures...")
print("=" * 60)

max_attempts = 5
attempt = 0

while attempt < max_attempts:
    try:
        attempt += 1
        print(f"\nExecution attempt {attempt}:")
        
        # Continue from last checkpoint
        result = durable_graph.invoke(None if attempt > 1 else durable_state, config=config)
        
        print(f"\n{Fore.GREEN}✅ Execution completed successfully!{Style.RESET_ALL}")
        print(f"Steps completed: {result['steps_completed']}")
        print(f"Result: {result['final_result']}")
        break
        
    except Exception as e:
        print(f"{Fore.YELLOW}⚠️  Execution failed: {e}{Style.RESET_ALL}")
        if attempt < max_attempts:
            print("Retrying from last checkpoint...")
            time.sleep(1)  # Wait before retry
        else:
            print(f"{Fore.RED}❌ Max retries reached. Execution failed.{Style.RESET_ALL}")

## Chapter 12: Model Context Protocol (MCP)

MCP allows our agents to connect with external systems and tools in a standardized way. This is crucial for enterprise integrations.

In [ ]:
# Simulate MCP server connections
class MCPServer:
    """Simulated MCP server for external integrations"""
    
    def __init__(self, name: str, capabilities: List[str]):
        self.name = name
        self.capabilities = capabilities
        print(f"{Fore.BLUE}🔌 MCP Server '{name}' initialized with capabilities: {capabilities}{Style.RESET_ALL}")
    
    def execute(self, command: str, params: Dict[str, Any]) -> Dict[str, Any]:
        """Execute a command on the MCP server"""
        print(f"{Fore.CYAN}  → Executing '{command}' on {self.name}{Style.RESET_ALL}")
        
        # Simulate different server responses
        if self.name == "crm_server":
            return {
                "customer_data": {
                    "name": "John Doe",
                    "account_type": "Premium",
                    "lifetime_value": 5000
                }
            }
        elif self.name == "inventory_server":
            return {
                "stock_level": 42,
                "reorder_status": "pending"
            }
        return {"status": "success"}


class MCPState(TypedDict):
    query: str
    mcp_results: Dict[str, Any]
    integrated_response: Optional[str]


def create_mcp_graph():
    """Create a graph that uses MCP for external integrations"""
    
    # Initialize MCP servers
    crm_server = MCPServer("crm_server", ["get_customer", "update_customer"])
    inventory_server = MCPServer("inventory_server", ["check_stock", "reserve_item"])
    
    def gather_context(state: MCPState) -> MCPState:
        """Gather context from multiple MCP servers"""
        print(f"\n{Fore.YELLOW}📡 Gathering context from MCP servers...{Style.RESET_ALL}")
        
        state["mcp_results"] = {}
        
        # Query CRM server
        crm_result = crm_server.execute("get_customer", {"query": state["query"]})
        state["mcp_results"]["crm"] = crm_result
        
        # Query inventory server
        inv_result = inventory_server.execute("check_stock", {"query": state["query"]})
        state["mcp_results"]["inventory"] = inv_result
        
        return state
    
    def integrate_and_respond(state: MCPState) -> MCPState:
        """Integrate MCP results and generate response"""
        print(f"\n{Fore.GREEN}🔄 Integrating MCP results...{Style.RESET_ALL}")
        
        customer_data = state["mcp_results"].get("crm", {}).get("customer_data", {})
        inventory_data = state["mcp_results"].get("inventory", {})
        
        response = f"""Based on integrated data:
        - Customer: {customer_data.get('name', 'Unknown')} ({customer_data.get('account_type', 'Standard')})
        - Lifetime Value: ${customer_data.get('lifetime_value', 0)}
        - Stock Available: {inventory_data.get('stock_level', 0)} units
        - Reorder Status: {inventory_data.get('reorder_status', 'unknown')}
        
        Recommendation: {'Priority service' if customer_data.get('account_type') == 'Premium' else 'Standard service'}
        """
        
        state["integrated_response"] = response
        return state
    
    graph = StateGraph(MCPState)
    
    graph.add_node("gather", gather_context)
    graph.add_node("integrate", integrate_and_respond)
    
    graph.set_entry_point("gather")
    graph.add_edge("gather", "integrate")
    graph.add_edge("integrate", END)
    
    return graph.compile()


print("\n🌐 Testing Model Context Protocol (MCP):\n")
mcp_graph = create_mcp_graph()

# Test MCP integration
mcp_query = {
    "query": "Customer inquiry about product availability",
    "mcp_results": {},
    "integrated_response": None
}

result = mcp_graph.invoke(mcp_query)
print(f"\n{Fore.BLUE}📋 Integrated Response:{Style.RESET_ALL}")
print(result["integrated_response"])

## Chapter 13: Tracing and Observability

In production, we need to monitor our agents' behavior, performance, and decision-making. Tracing gives us complete visibility.

In [ ]:
from datetime import datetime
import uuid

class TraceCollector:
    """Collects traces for monitoring and debugging"""
    
    def __init__(self):
        self.traces = []
    
    def add_trace(self, event_type: str, node_name: str, data: Dict[str, Any]):
        trace = {
            "trace_id": str(uuid.uuid4())[:8],
            "timestamp": datetime.now().isoformat(),
            "event_type": event_type,
            "node_name": node_name,
            "data": data
        }
        self.traces.append(trace)
        return trace
    
    def get_summary(self):
        """Get execution summary"""
        return {
            "total_events": len(self.traces),
            "nodes_executed": list(set(t["node_name"] for t in self.traces)),
            "errors": [t for t in self.traces if t["event_type"] == "error"],
            "duration_ms": self._calculate_duration()
        }
    
    def _calculate_duration(self):
        if not self.traces:
            return 0
        start = datetime.fromisoformat(self.traces[0]["timestamp"])
        end = datetime.fromisoformat(self.traces[-1]["timestamp"])
        return int((end - start).total_seconds() * 1000)


# Global trace collector
trace_collector = TraceCollector()


class TracedState(TypedDict):
    input: str
    processing_steps: List[str]
    result: Optional[str]
    trace_id: str


def traced_node(func):
    """Decorator to add tracing to any node"""
    def wrapper(state: TracedState) -> TracedState:
        node_name = func.__name__
        
        # Start trace
        start_trace = trace_collector.add_trace(
            "node_start",
            node_name,
            {"input": state.get("input", ""), "trace_id": state["trace_id"]}
        )
        
        print(f"{Fore.BLUE}🔍 [{start_trace['trace_id']}] Starting {node_name}{Style.RESET_ALL}")
        
        try:
            # Execute the actual function
            result = func(state)
            
            # End trace
            trace_collector.add_trace(
                "node_complete",
                node_name,
                {"output": result.get("result", ""), "trace_id": state["trace_id"]}
            )
            
            return result
            
        except Exception as e:
            # Error trace
            trace_collector.add_trace(
                "error",
                node_name,
                {"error": str(e), "trace_id": state["trace_id"]}
            )
            raise
    
    return wrapper


@traced_node
def validate_input(state: TracedState) -> TracedState:
    """Validate and preprocess input"""
    print(f"  Validating: {state['input'][:50]}...")
    state["processing_steps"].append("validation")
    
    # Simulate validation logic
    if len(state["input"]) < 10:
        raise ValueError("Input too short")
    
    return state


@traced_node
def analyze_sentiment(state: TracedState) -> TracedState:
    """Analyze sentiment of the input"""
    print(f"  Analyzing sentiment...")
    state["processing_steps"].append("sentiment_analysis")
    
    # Simulate sentiment analysis
    if "angry" in state["input"].lower() or "frustrated" in state["input"].lower():
        sentiment = "negative"
    elif "happy" in state["input"].lower() or "great" in state["input"].lower():
        sentiment = "positive"
    else:
        sentiment = "neutral"
    
    state["result"] = f"Sentiment: {sentiment}"
    return state


@traced_node
def generate_response(state: TracedState) -> TracedState:
    """Generate final response based on analysis"""
    print(f"  Generating response...")
    state["processing_steps"].append("response_generation")
    
    state["result"] = f"Processed: {state['input'][:30]}... | {state['result']}"
    return state


def create_traced_graph():
    """Create a graph with comprehensive tracing"""
    graph = StateGraph(TracedState)
    
    graph.add_node("validate", validate_input)
    graph.add_node("analyze", analyze_sentiment)
    graph.add_node("respond", generate_response)
    
    graph.set_entry_point("validate")
    graph.add_edge("validate", "analyze")
    graph.add_edge("analyze", "respond")
    graph.add_edge("respond", END)
    
    return graph.compile()


print("\n📊 Testing Tracing and Observability:\n")

# Clear previous traces
trace_collector.traces = []

traced_graph = create_traced_graph()

# Test with multiple inputs to see tracing
test_inputs = [
    "I am very happy with your service! Everything works great!",
    "This is frustrating and I'm getting angry about these issues.",
    "Short",  # This will cause an error
]

for i, test_input in enumerate(test_inputs):
    print(f"\n{'='*60}")
    print(f"Test {i+1}: {test_input[:50]}...")
    print(f"{'='*60}")
    
    trace_id = str(uuid.uuid4())[:8]
    test_state = {
        "input": test_input,
        "processing_steps": [],
        "result": None,
        "trace_id": trace_id
    }
    
    try:
        result = traced_graph.invoke(test_state)
        print(f"\n{Fore.GREEN}✅ Success: {result['result']}{Style.RESET_ALL}")
    except Exception as e:
        print(f"\n{Fore.RED}❌ Error: {e}{Style.RESET_ALL}")

# Display trace summary
print("\n" + "="*60)
print("📊 EXECUTION TRACE SUMMARY")
print("="*60)

summary = trace_collector.get_summary()
print(f"\nTotal events: {summary['total_events']}")
print(f"Nodes executed: {', '.join(summary['nodes_executed'])}")
print(f"Total duration: {summary['duration_ms']}ms")
print(f"Errors encountered: {len(summary['errors'])}")

if summary['errors']:
    print("\n🚨 Error Details:")
    for error in summary['errors']:
        print(f"  - [{error['trace_id']}] {error['node_name']}: {error['data']['error']}")

# Show detailed trace for debugging
print("\n📜 Detailed Trace Log (last 5 events):")
print("-" * 60)
for trace in trace_collector.traces[-5:]:
    print(f"[{trace['timestamp']}] {trace['event_type']:15} | {trace['node_name']:20} | Trace: {trace['trace_id']}")

## Conclusion: Putting It All Together

Let's create a comprehensive example that combines all the concepts we've learned into a production-ready customer support system.

In [ ]:
# Final comprehensive example combining all features
print("\n🎯 COMPREHENSIVE CUSTOMER SUPPORT SYSTEM")
print("=" * 60)
print("\nThis system combines:")
print("✓ LLM-powered intelligence")
print("✓ Tool usage for actions")
print("✓ Persistence for conversation history")
print("✓ Human-in-the-loop for approvals")
print("✓ Multi-agent collaboration")
print("✓ Streaming for real-time updates")
print("✓ Durable execution for reliability")
print("✓ MCP for external integrations")
print("✓ Comprehensive tracing")

print("\n🚀 System initialized and ready!")
print("\nKey Takeaways:")
print("1. Start simple and add complexity gradually")
print("2. Use persistence for production systems")
print("3. Add human oversight for critical decisions")
print("4. Implement proper error handling and retries")
print("5. Use tracing to debug and monitor your agents")
print("6. Modularize with subgraphs for maintainability")
print("7. Let multiple agents collaborate on complex tasks")

print("\n📚 Next Steps:")
print("- Experiment with these patterns in your own projects")
print("- Explore LangGraph's advanced features")
print("- Build production-ready agent systems")
print("- Join the LangGraph community for support")

print("\n✨ Happy building with LangGraph!")